In [ ]:
%matplotlib inline

# Visualisation
import matplotlib.pyplot as plt

# monitoring
import time

# data cleaning
import re

# lemmatisation
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


# intermediate storage
import csv
import pandas as pd
import pickle

# stats
import numpy as np

# stopwords
from nltk.corpus import stopwords

# Clustering
from scipy.spatial.distance import squareform
from scipy.cluster import hierarchy

# Machine learning
import sklearn
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import roc_curve, auc
from matplotlib.legend_handler import HandlerLine2D

In [ ]:
ngram_length = 2
min_yearly_df = 5   

long_ma_length = 7   # 12
short_ma_length = 4  #6
signal_line_ma = 2   #3
significance_ma_length = 2  # actual 3

significance_threshold = 0.0002 # actual 0.0002
years_above_significance = 3
testing_period = 3

# Detection threshold is set such that the top 500 terms are chosen
burstiness_threshold_prediction = 0.003
burstiness_threshold_detection = 0.000020

plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rc('font', family='sans-serif')

year_range = list(range(2014,2018))

In [ ]:
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
htmltags = '<[^>]+>'
htmlspecial = '&#?[xX]?[a-zA-Z0-9]{2,8};'

start_delimiter = 'documentstart'

sent_delimiter = 'sentenceboundary'
end_delimiter = 'documentend'

delimiters = [start_delimiter, sent_delimiter, end_delimiter]

# Download the lemmatisesr
wnl = WordNetLemmatizer()

# Create a tokeniser
count = CountVectorizer(strip_accents='ascii', min_df=1)
tokeniser = count.build_analyzer()

def normalise_acronymns(text):
    '''
    Remove the periods in acronyms. 
    Adapted from the method found at https://stackoverflow.com/a/40197005 
    '''
    return re.sub(r'(?<!\w)([A-Z, a-z])\.', r'\1', text)

def normalise_decimals(text):
    '''
    Remove the periods in decimal numbers and replace with POINT
    '''
    return re.sub(r'([0-9])\.([0-9])', r'\1POINT\2', text)

def split_into_sentences(text):
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    
    # my addition
    text = re.sub(htmltags, " ", text)
    text = re.sub(htmlspecial, " ", text)
    
    if "Ph.D" in text: 
        text = text.replace("Ph.D.","PhD")
        
    text = re.sub("\s" + alphabets + "[.] "," \\1",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1\\2\\3",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1\\2",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1 \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1",text)
    text = re.sub(" " + alphabets + "[.]"," \\1",text)
    
    if "”" in text: 
        text = text.replace(".”","”.")
    if "\"" in text: 
        text = text.replace(".\"","\".")
    if "!" in text: 
        text = text.replace("!\"","\"!")
    if "?" in text: 
        text = text.replace("?\"","\"?")
        
    text = text.replace(".","<stop>")
    text = text.replace("?","<stop>")
    text = text.replace("!","<stop>")
    
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    
    non_empty = []
    for s in sentences: 
         # we require that there be two alphanumeric characters in a row
        if len(re.findall("[A-Za-z0-9][A-Za-z0-9]", s)) > 0:
            non_empty.append(s)
    return non_empty

def pad_sentences(sentences):
    '''
    Takes a list of sentences and returns a string in which:
        - The beginning of the abstract is indicated by DOCUMENTSTART
        - The end is indicated by DOCUMENTEND
        - Sentence boundaries are indicated by SENTENCEBOUNDARY
        
    The number of delimiters used is dependent on the ngram length
    '''
    sent_string = (' '+(sent_delimiter+' ')*(ngram_length-1)).join(sentences)
    
    return (start_delimiter+' ')*(ngram_length-1) + sent_string + (' '+end_delimiter)*(ngram_length-1)
    
def cleaning_pipeline(title, abstract):
    '''
    Takes a binary string and returns a list of cleaned sentences, stripped of punctuation and lemmatised
    '''

    title = normalise_decimals(normalise_acronymns(title.decode()))
    abstract = normalise_decimals(normalise_acronymns(abstract.decode()))
    sentences = [title] + split_into_sentences(abstract)
    
    # strip out punctuation and make lowercase
    clean_sentences = []
    for s in sentences:
        
        # Deal with special cases
        s = re.sub(r'[-/]', ' ', s)
        
        # Remove all other punctuation
        s = re.sub(r'[^\w\s]','',s)
        clean_sentences.append(s.lower())
        
    # pad sentences with delimiters
    text = pad_sentences(clean_sentences)
    
    # Lemmatise word by word
    lemmas = []
    for word in tokeniser(text):
        lemmas.append(wnl.lemmatize(word))
        
    return ' '.join(lemmas)


def calc_macd(dataset):
    long_ma = dataset.ewm(span=long_ma_length).mean()
    short_ma = dataset.ewm(span=short_ma_length).mean()
    significance_ma = dataset.ewm(span=significance_ma_length).mean()
    macd = short_ma - long_ma
    signal = macd.ewm(span=signal_line_ma).mean()
    hist = macd - signal
    return long_ma, short_ma, significance_ma, macd, signal, hist

def calc_significance(stacked_vectors, significance_threshold, n):
    # Must have been above the significance threshold for two consecutive timesteps
    a = stacked_vectors > significance_threshold   
    b = a.rolling(window = n).sum()
    return stacked_vectors[stacked_vectors.axes[1][np.where(b.max()>=n)[0]]]
    
def calc_burstiness(hist, scaling_factor):
    return hist.iloc[long_ma_length-1:]/scaling_factor

def calc_scaling(significance_ma, method):
    if method == "max":
        scaling = significance_ma.iloc[significance_ma_length-1:].max()
    elif method == "mean":
        scaling = significance_ma.iloc[significance_ma_length-1:].mean()
    elif method == "sqrt":
        scaling = np.sqrt(significance_ma.iloc[significance_ma_length-1:].max()  )      
    return scaling
def max_burstiness(burstiness, absolute=False):
    if absolute:
        b = pd.concat([np.abs(burstiness).max(), burstiness.idxmax()], axis=1)# actual axis = 1
    else:
        b = pd.concat([burstiness.max(), burstiness.idxmax()], axis=1)
    b.columns = ["max", "location"]
    return b

def feature_selection(dataset):
    '''
    Compile the features for the prediction step
    '''
    long_ma = dataset.ewm(span=long_ma_length).mean()
    short_ma = dataset.ewm(span=short_ma_length).mean()
    significance_ma = dataset.ewm(span=significance_ma_length).mean()
    macd = short_ma - long_ma
    signal = macd.ewm(span=signal_line_ma).mean()
    hist = macd - signal
    
    scaling_factor = calc_scaling(significance_ma, "sqrt")
    burstiness_over_time = calc_burstiness(hist, scaling_factor)
    burstiness = max_burstiness(burstiness_over_time)
    
    
    X = long_ma.iloc[long_ma_length:].T
    scaled_hist = hist.iloc[long_ma_length:]/scaling_factor
    scaled_signal = signal.iloc[long_ma_length:]/scaling_factor
    
    Xtra = pd.concat([significance_ma.iloc[-1], 
                      dataset.iloc[-1],
                        significance_ma.iloc[significance_ma_length:].std()/scaling_factor,
                        significance_ma.iloc[significance_ma_length:].max(),
                        significance_ma.iloc[significance_ma_length:].min(),
                      scaling_factor
                        ], axis=1)
    X = pd.concat([X,scaled_hist.T,scaled_signal.T,Xtra], axis=1)

    X.columns = [str(i) for i in range(8)] + ["hist"+str(i) for i in range(8)] + ["signal"+str(i) for i in range(8)] + [
          "significance",
                        "prevalence",
                        "scaled std",
                        "max",
                        "min",
                        "scaling"
                    ]


    return X

def balanced_subsample(x,y,subsample_size=1.0):
    # from https://stackoverflow.com/a/23479973
    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            np.random.shuffle(this_xs)

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)
        xs.append(x_)
        ys.append(y_)

    xs = np.concatenate(xs)
    ys = np.concatenate(ys)

    return xs,ys

In [ ]:
#df = pd.read_csv("C:/drive/new/Institute/ACADEMY OF SCIENTIFIC AND INNOVATIVE RESEARCH ACSIR.csv",encoding='latin-1')
column = ['PY','SC','WC','DE']
df = pd.read_csv("C:/drive/all_merge.csv", usecols = column,low_memory=True)
df1 = df[['PY','WC']]
df1.to_csv("C:/drive/mergenew.csv")
df1 = df1.dropna()
year = [2014,2015,2016,2017,2018]
df1 = df1[df1['PY'].isin(year)] 
df1.shape
print(df['PY'].unique())
# df1 = df[['PD','PT']] (correct but not used right now)
df1.groupby(df1['PY']).size().plot(kind='bar')
#df1.groupby(df1['PD']).size().plot(kind='bar') # wrong data in file PY saved as PD
plt.title(" Documnets per Year")
plt.xlabel("Year")
plt.ylabel("Number of documnets")
plt.grid(True)


### The average length of titles and abstracts

In [ ]:
df1.head(5)

In [ ]:
df
dftext = df[['TI','AB','ID','DE']]
#dftext['TI'].value_counts()[:30].plot(kind='barh')
dftext['TI'].value_counts()[:10].plot(kind='barh')
dftext['ID'].value_counts()[:10].plot(kind='barh')
dftext['AB'].value_counts()[:10].plot(kind='barh')
dftext['DE'].value_counts()[:10].plot(kind='barh')

In [ ]:
stop = set(stopwords.words('english'))
stop = set([s.replace("'", "") for s in stop])

# Add years to prevent spikes
for year in range(2014, 2019):
    stop.add(str(year))

# Add small numbers
for num in range(0, 100):
    if len(str(num)) < 2:
        stop.add(str(num))
        num = '0' + str(num)
        
    stop.add(str(num))
    
# Add these extra stopwords to the list
extra = [
    'use', 'using', 'uses', 'used', 'based', 'including', 'include', 'approach',
    'wa', 'ha', 'doe'
        ]
for word in extra:
    stop.add(word)

In [ ]:
#dft = dft.sort_values(['PY','AB'],ascending = True)
df1 = df1.sort_values(['PY','DE'],ascending = True) # PY==PD ,AB ==ID

#for y in range(1989,2009):
for y in range(2014,2018):
    #dd = dft.loc[dft['PY'] == y,['PY','TI']] 
    
    dd = df1.loc[df1['PY'] == y,['PY','DE']] 
    print(dd)


### Build a vocabulary

We have to build a vocabulary before we vectorise the data. This is because we want to set limits on the size of the vocabulary.

Terms must occur at least 5 times in at least one year. This removes one-off spelling errors or excessively rare terms, which, given the intended application, are not interesting to us.
We take uni, bi and tri-grams.
We use sentence delimeters to avoid taking bi and tri-grams across sentence boundaries.

In [ ]:
# used when three field data is taken 
cv=CountVectorizer()
sample = pd.DataFrame(df1['DE']+','+df1['ID'], columns=['Output'])
word_count_vector=cv.fit_transform(sample['Output'])
print(word_count_vector.shape)

 



In [ ]:
vocab = set()
df1 = df1.sort_values(['PY','WC',],ascending = True)
for y in range(2014, 2019):
    
    dd = df1.loc[df1['PY'] == y,['PY','WC']] 
    #dd = dft.loc[dft['PD'] == y,['PD','CA']] 
    #sample = pd.DataFrame(df1['DE']+','+df1['ID']+','+df1['TI'], columns=['Output'])
    
    t0 = time.time()
   
 #     vectorizer = CountVectorizer(strip_accents='ascii', 
#                                   stop_words='english',
#                                  ngram_range=(1,3), 
#                                  min_df=5,  max_features=20000)
    vectorizer = CountVectorizer(strip_accents ='ascii', 
                                 stop_words='english',
                                 ngram_range=(1,3), 
                                 min_df=30,max_features=200000)  
#    vector = vectorizer.fit_transform(sample['Output'])# .values.settype('U') for this we convert to unicode if not 
    vector = vectorizer.fit_transform(dd.WC)                                           # this can run without it 
    #print(vector.toarray())
    # Save the new words
    vocab = vocab.union(vectorizer.vocabulary_.keys())
    #print(vocab)
    print(y, len(vocab), time.time()-t0)

vocabulary = {}
i = 0
for v in vocab:
    # Remove delimiters
    if start_delimiter in v:
        pass
    elif end_delimiter in v:
        pass
    elif sent_delimiter in v:
        pass
    else:
        vocabulary[v] = i
        i += 1
        
print(len(vocabulary.keys()))    


### Go year by year and vectorise based on our vocabulary
We read in the cleaned data and vectorise it according to our vocabulary.

In [ ]:
vectors = []
df1 = df1.sort_values(['PY','WC'],ascending = True) # PY==PD ,AB ==ID ,TI ==CA

#for y in range(1989,2009):
for y in range(2014,2019):
    dd = df1.loc[df1['PY'] == y,['PY','WC']] 
    #dd = df1.loc[df1['PY'] == y,['PD','CA']] 
    
    # The same as above, applied year by year instead.
    t0 = time.time()


    vectorizer = CountVectorizer(strip_accents='ascii', 
                                ngram_range=(1,3),
                                stop_words='english',
                                vocabulary=vocabulary)
                                
    #vectorizer = CountVectorizer(ngram =(1,3) ,stop_words = stop ,vocabulary = vocabulary)
  
    vectors.append(vectorizer.fit_transform(dd.WC))
    #print(vectors)
    
    print(y, time.time()-t0)

In [ ]:
summed_vectors = []
for y in range(len(vectors)):
    
    vector = vectors[y]
    
    # Set all elements that are greater than one to one -- we do not care if a word is used multiple times in 
    # the same document
    vector[vector>1] = 1

    # Sum the vector along columns
    summed = np.squeeze(np.asarray(np.sum(vector, axis=0)))
   
    normalised = summed/vector.shape[0]
    summed_vectors.append(normalised)
print("stacked_transpose")    
stacked_vectors = np.stack(summed_vectors, axis=1)
print(stacked_vectors)
print(stacked_vectors.transpose().shape)

### Summing the vectors
We sum the vectors along columns, so that we have the popularity of each term in each year.

All >1 elements are set to 1. This is because it does not matter for our application if a word is used multiple times in an abstract.
We divide by the number of documents in each year to normalise the score. Therefore, the 1988 column is divided by ~6000, etc

In [ ]:
summed_vectors = []
for y in range(len(vectors)):
    
    vector = vectors[y]
    
    # Set all elements that are greater than one to one -- we do not care if a word is used multiple times in 
    # the same document
    #vector[vector>1] = 1

    # Sum the vector along columns
    summed = np.squeeze(np.asarray(np.sum(vector, axis=0)))
    
    # Normalise by dividing by the number of documents in that year
    normalised = summed/vector.shape[0]
    
    # Save the summed vector
    summed_vectors.append(normalised)
    
     #stack vectors vertically, so that we have the full history of popularity/time for each term
stacked_vectors = np.stack(summed_vectors, axis=1)

#stacked_vectors = np.stack(summed_vectors)
print(stacked_vectors.shape)

stacked_vectors=pd.DataFrame(stacked_vectors.transpose(), columns=list(vocabulary.keys()))
print(stacked_vectors)
#stacked_vectors.to_csv("C:\drive\stacked_vectors.csv")

In [ ]:
pickle.dump(vectors, open('C:/drive/burst/stacked_vectors.p', "wb"))

In [ ]:
# vectorise again, using these terms only for the bursty vectors
vectors = []
for year in range(2014, 2019):

# The same as above, applied year by year instead.
    t0 = time.time()
    vectorizer = CountVectorizer(strip_accents='ascii',ngram_range=(1,3),
                                  stop_words='english', vocabulary=bursts)
    vector = vectorizer.fit_transform(df1.WC)
# If any element is larger than one, set it to one
    vector.data = np.where(vector.data>0, 1, 0)
    vectors.append(vector)
pickle.dump(vectors, open('C:/drive/burst/burstvectors_500.p', "wb"))

In [ ]:
stacked_vectors = pickle.load(open('C:/drive/burst/stacked_vectors.p', "rb"))
burstvectors = pickle.load(open('C:/drive/burst/burstvectors_500.p', "rb"))
bursts = pickle.load(open('C:/drive/burst/bursts.p', "rb"))
clusters = pickle.load(open('C:/drive/burst/clusters.p', "rb"))
stackedvectors = pickle.load(open('C:/drive/burst/stackedvectors.p', "rb"))

In [ ]:
print(stackedvectors)

In [ ]:
stacked_vectors1 = pd.DataFrame(stackedvectors)
stacked_vectors1.to_csv("C:/drive/burst/stacked_vectorsfinal.csv")

In [ ]:
ml_list = []
for list in stacked_vectors:
    ml_list.append(list)
stacked_vectors2 = pd.DataFrame(ml_list)
stacked_vectors2.to_csv("C:/drive/burst/stacked_vectors3.csv")

In [ ]:
ml_list = []
for list in stackedvectors:
    input()
    print(list)
    for l in list:
        print(l)
        ml_list.append(l)
    
# stacked_vectors1 = pd.DataFrame(stacked_vectors)
# stacked_vectors1.to_csv("C:/drive/burst/stacked_vectorsnew.csv")

In [ ]:
for row in stacked_vectors : 
    print(row) 
rez = [[stacked_vectors[j][i] for j in range(len(stacked_vectors))] for i in range(len(stacked_vectors[0]))] 
print("\n") 
for row in rez: 
    print(row) 

### Normalise the vectors again
The number of tokens per abstract has changed over time. We now normalise again, so each year sums to 100.

In [ ]:
# Not required in my case as I am not using Abstract NO RRRUn
normalisation = stacked_vectors.sum(axis=1)
print(normalisation)
stacked_vectors = stacked_vectors.divide(normalisation, axis='index')*100
print(stacked_vectors)

### Apply a significance threshold
We require that each term has been above a given significance threshold for 3 years. This shortens the vocabulary and 
removes single year spikes due to anomalous events.

In [ ]:
# Not used in my case no RRUN
stacked_vectors = calc_significance(stacked_vectors, significance_threshold, years_above_significance)
print(stacked_vectors.shape)
print(stacked_vectors)

In [ ]:
pickle.dump(vectors, open('C:/drive/burst/stackedvectors.p', "wb"))

In [ ]:
long_ma, short_ma, significance_ma, macd, signal, hist = calc_macd(stacked_vectors)
scaling_factor = calc_scaling(significance_ma, "max")
# print(scaling_factor)
burstiness_over_time = calc_burstiness(hist,False)
print(burstiness_over_time)


## Calculate burstiness

In [ ]:
long_ma, short_ma, significance_ma, macd, signal, hist = calc_macd(stacked_vectors)

# scaling_factor = calc_scaling(significance_ma, "max")
# print(scaling_factor)
# burstiness_over_time = calc_burstiness(hist, scaling_factor)
burstiness_over_time = hist

#burstiness_over_time = calc_burstiness(hist, inverse)
# print("burstiness_over_time:")
# print(burstiness_over_time.shape)




#print(burstiness_over_time[b])
burstiness = max_burstiness(burstiness_over_time,True)
#burstiness = abs(burstiness_over_time)
print(burstiness)


### Set a threshold such that the top 500 bursty terms are included

In [ ]:
 print(np.sum(burstiness["max"]>0.000024))


In [ ]:
#bursts = list(burstiness["max"].index[np.where(burstiness["max"]>burstiness_threshold_detection)[0]])
bursts = list(burstiness["max"].index[np.where(burstiness["max"]>0.000024)[0]])
print(bursts)

In [ ]:
pickle.dump(bursts, open('C:/drive/burst/bursts.p', "wb"))

In [ ]:
bursts1 = pd.DataFrame(bursts)
bursts1.to_csv("C:/drive/burst/bursts2.csv")

### Cluster Based on Co-occurence

In [ ]:
# vectorise again, using these terms only
vectors = []
df1 = df1.sort_values(['PY','WC'],ascending = True) # PY==PD ,AB ==ID ,TI ==CA

#for y in range(1989,2009):
for y in range(2014,2019):
    dd = df1.loc[df1['PY'] == y,['PY','WC']] 
    #dd = df1.loc[df1['PY'] == y,['PD','CA']] 
    
    # The same as above, applied year by year instead.
    t0 = time.time()


    vectorizer = CountVectorizer(strip_accents='ascii', 
                                ngram_range=(1,3),
                                stop_words='english',
                                vocabulary=vocabulary)
                                

    vector = vectorizer.fit_transform(dd.WC)
    
    # If any element is larger than one, set it to one
    vector.data = np.where(vector.data>0, 1, 0)
    
    vectors.append(vector)
    
    print(y, time.time()-t0)

In [ ]:
i=0
cooccurrence = []
for v in vectors:
    i = i+1
    c = v.T*v
    c.setdiag(0)
    c = c.todense()
    cooccurrence.append(c)
all_cooccurrence = np.sum(cooccurrence, axis=0)

# Translate co-occurence into a distance
dists = 1- all_cooccurrence/all_cooccurrence.max()

# Remove the diagonal (squareform requires diagonals be zero)
dists -= np.diag(np.diagonal(dists))

# Put the distance matrix into the fṁormat required by hierachy.linkage
flat_dists = squareform(dists)

# Get the linkage matrix
linkage_matrix = hierarchy.linkage(flat_dists, "ward")

assignments = hierarchy.fcluster(linkage_matrix, 120, 'maxclust')

print(len(bursts))
print(len(set(assignments)))

clusters = defaultdict(list)

for term, assign in zip(bursts, assignments):
    clusters[assign].append(term)
    
print(clusters)

for key in sorted(clusters.keys()):
    print(key, ':',  ', '.join(clusters[key]))

In [ ]:
pickle.dump(clusters, open('C:/drive/burst/clusters.p', "wb"))

In [ ]:
cluster = pd.DataFrame(clusters)
cluster.to_csv('C:/drive/burst/clusters.csv')

### Graph selected bursty terms over time
We manually remove clusters that contain copyright declarations, etc. Then we filter down to 52, choosing 
a representative sample over time. We choose one or two terms to represent each cluster.

In [ ]:
df1 = pd.read_csv('C:\\drive\\burst\\bursts.csv')
df1.head(5)
clusters = [d.split(', ') for d in df1['terms value']]

In [ ]:
clusters 

In [ ]:
pickle.dump(vectors, open('C:/drive/burst/bursts.p', "wb"))

In [ ]:
def get_prevalence(cluster):
    indices = []
    for term in cluster:
        indices.append(bursts.index(term))
        print(indices)
    prevalence = []
    for year in range(5):
        prevalence.append(100*np.sum(np.sum(stacked_vectors[year][:,indices], axis=1)>0)/stacked_vectors[year].shape[0])
        print(prevalence)
    return prevalence


yplots = 13
xplots = 4
fig, axs = plt.subplots(yplots, xplots)
plt.subplots_adjust(right=1, hspace=0.9, wspace=0.3)
plt.suptitle('Prevalence of selected bursty clusters over time', fontsize=14)
fig.subplots_adjust(top=0.95)
fig.set_figheight(16)
fig.set_figwidth(12)
x = np.arange(0,5)
prevalences = []
for i, cluster in enumerate(clusters):
    print(cluster)
    prevalence = get_prevalence(cluster)
    prevalences.append(prevalence)
#     title = df1.name[i]
    title = df1['name'][i]
    axs[int(np.floor((i/xplots)%yplots)), i%xplots].plot(x, prevalence, color='k', ls='-', label=title)
    axs[int(np.floor((i/xplots)%yplots)), i%xplots].grid()
    ymax=np.ceil(max(prevalence)*2)/2
    if ymax == 0.5 and max(prevalence) <0.25:
        ymax=0.25
    elif ymax == 2.5:
        ymax=3
    axs[int(np.floor((i/xplots)%yplots)), i%xplots].set_ylim(0,ymax)
    axs[int(np.floor((i/xplots)%yplots)), i%xplots].set_xlim(0,30)
    axs[int(np.floor((i/xplots)%yplots)), i%xplots].set_title(title, fontsize=12)
    
    
    if i%yplots != yplots-1:
        axs[i%yplots, int(np.floor((i/yplots)%xplots))].set_xticklabels([])
    else:
        axs[i%yplots, int(np.floor((i/yplots)%xplots))].set_xticklabels([1988, 1998, 2008, 2018])
        
axs[6,0].set_ylabel('Percentage of documents containing term (%)', fontsize=12)

### Prediction 

In [ ]:
development_data = {}
year_range = [2014,2015,2016,2017,2018]
for year in range(2014, 2019):
    year_idx = year_range.index(year)
    print(year_idx)
    # Use our three-year method to calc significance
    valid_vectors = calc_significance(stacked_vectors[:year_idx], significance_threshold, 1)

In [ ]:
development_data = {}
year_range = [2014,2015,2016,2017,2018]
for year in range(2014, 2019):
    year_idx = year_range.index(year)
    burstsnew = stackedvectors.keys()[bursts]
    
    # Create a new, much smaller dataset
    dataset = stackedvectors[burstsnew].iloc[:year_idx+1]
    
    # Get the scaled y values
    if year < 2018:
        y = stackedvectors[bursts].iloc[year_idx]
# Select features and store the data
    development_data[year] = {}
    development_data[year]["X"] = feature_selection(dataset)
    if year < 2018:
        development_data[year]["y"]=y-development_data[year]["X"]['significance']
    print(year, len(bursts))

In [ ]:
    
    # Recalculate the macd things based on this more limited dataset
    long_ma, short_ma, significance_ma, macd, signal, hist = calc_macd(stacked_vectors)
    
    
    # Calculate scaling factor
    scaling_factor = calc_scaling(significance_ma.iloc[max(long_ma_length, year_idx-19):year_idx+1], "sqrt")

    # Calculate the burstiness
    burstiness_over_time = calc_burstiness(hist, scaling_factor)
    burstiness = max_burstiness(burstiness_over_time)

    # Choose terms that are above both thresholds (burstiness, and also most recent year was significant)
    burst_idx = np.where((burstiness["max"]>0.000024)&(significance_ma.iloc[year_idx]>significance_threshold))[0]
    
    # Find the actual names of these terms
    bursts = valid_vectors.keys()[burst_idx]
    
    # Create a new, much smaller dataset
    dataset = stacked_vectors[bursts].iloc[:year_idx+1]
    
    # Get the scaled y values
    if year < 2018:
        y = stacked_vectors[bursts].iloc[year_idx]
# Select features and store the data
    development_data[year] = {}
    development_data[year]["X"] = df1['PY']
    if year < 2018:
        development_data[year]["y"]=y-development_data[year]["X"]['WC']
    print(year, len(bursts))

In [ ]:
dataset = {}

for year in stacked_vectors:
    for item in year:
        print(item)

In [ ]:
    
    # Get the scaled y values
    if year < 2018:
        y = stacked_vectors[bursts].iloc[year_idx]
# Select features and store the data
    development_data[year] = {}
    development_data[year]["X"] = df1['PY']
    if year < 2018:
        development_data[year]["y"]=y-development_data[year]["X"]['WC']
    print(year, len(bursts))

In [ ]:
development_data = {}
year_range = [2014,2015,2016,2017,2018]
for year in range(2014, 2019):
    year_idx = year_range.index(year)
    
    
    # Get the scaled y values
    if year < 2018:
        y = stacked_vectors[bursts].iloc[year_idx]
# Select features and store the data
    development_data[year] = {}
    development_data[year]["X"] = df1['PY']
    if year < 2018:
        development_data[year]["y"]=y-development_data[year]["X"]['WC']
    print(year, len(bursts))
    

In [ ]:
df2 = pd.read_csv("C:\drive\stacked_vectors.csv")

In [ ]:
df2.list()

## Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
column = ['PY', 'WC']
df1 = pd.read_csv("C:/drive/all_merge.csv", usecols = column,low_memory=True)
X=df1[['PY']]  # Features
y=df1['WC']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=1000)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [ ]:

for y in range(2014,2019):
    dd = df1.loc[df1['PY'] == y,['PY','WC']]     
    cv = CountVectorizer(strip_accents ='ascii', 
                     ngram_range=(1,3),min_df=30,
                     max_features=200000)  
    count_vector=cv.fit_transform(dd.WC)
    vocab = vocab.union(vectorizer.vocabulary_.keys())
    #print(vocab)
    print(y, len(vocab), time.time()-t0)

vocabulary = {}
i = 0
for v in vocab:
    # Remove delimiters
    if start_delimiter in v:
        pass
    elif end_delimiter in v:
        pass
    elif sent_delimiter in v:
        pass
    else:
        vocabulary[v] = i
        i += 1
        
print(len(vocabulary.keys()))    


In [ ]:
cv.vocabulary_

In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
development_data = {}
for year in range(2014, 2019):
    year_idx = year_range.index(year)
    
    # Use our three-year method to calc significance
    valid_vectors = calc_significance(df2[:year_idx+1], significance_threshold, 1)
    
    # Recalculate the macd things based on this more limited dataset
    long_ma, short_ma, significance_ma, macd, signal, hist = calc_macd(valid_vectors)
    
    
    # Calculate scaling factor
    scaling_factor = calc_scaling(significance_ma.iloc[max(long_ma_length, year_idx-19):year_idx+1], "sqrt")

    # Calculate the burstiness
    burstiness_over_time = calc_burstiness(hist, scaling_factor)
    burstiness = max_burstiness(burstiness_over_time)

    # Choose terms that are above both thresholds (burstiness, and also most recent year was significant)
    burst_idx = np.where((burstiness["max"]>0.0012)&(significance_ma.iloc[year_idx]>significance_threshold))[0]
    
    # Find the actual names of these terms
    bursts = valid_vectors.keys()[burst_idx]
    
    # Create a new, much smaller dataset
    dataset = stacked_vectors[bursts].iloc[year_idx-19:year_idx+1]
    
    # Get the scaled y values
    if year < 2015:
        y = stacked_vectors[bursts].iloc[year_idx+testing_period]


    # Select features and store the data
    development_data[year] = {}
    development_data[year]["X"] = feature_selection(dataset)
    if year < 2015:
        development_data[year]["y"]=y-development_data[year]["X"]['significance']
    print(year, len(bursts))

In [ ]:
scores = {}
for threshold in np.arange(0.0006, 0.0017, 0.0002):
    scores[threshold] = {}
    for year in range(2008, 2013):
        year_idx = year_range.index(year)

        # Use our three-year method to calc significance
        valid_vectors = calc_significance(stacked_vectors[:year_idx+1], significance_threshold, 3)

        # Recalculate the macd things based on this more limited dataset
        long_ma, short_ma, significance_ma, macd, signal, hist = calc_macd(valid_vectors)


        # Calculate scaling factor
        scaling_factor = calc_scaling(significance_ma.iloc[max(long_ma_length, year_idx-19):year_idx+1], "sqrt")

        # Calculate the burstiness
        burstiness_over_time = calc_burstiness(hist, scaling_factor)
        burstiness = max_burstiness(burstiness_over_time)

        # Choose terms that are above both thresholds (burstiness, and also most recent year was significant)
        burst_idx = np.where((burstiness["max"]>threshold)&(significance_ma.iloc[year_idx]>significance_threshold))[0]

        # Find the actual names of these terms
        bursts = valid_vectors.keys()[burst_idx]

        # Create a new, much smaller dataset
        dataset = stacked_vectors[bursts].iloc[year_idx-19:year_idx+1]

        # Select features and store the data
        development_data[year] = {}
        development_data[year]["X"] = feature_selection(dataset)
        
        development_data[year]["y"] = {}
        
        for interval in range(1,6):
            # Get the scaled y values
            y = stacked_vectors[bursts].iloc[year_idx+interval]
            development_data[year]["y"][interval]=y-development_data[year]["X"]['significance']
    
    
    X = np.array(pd.concat([development_data[year]["X"] for year in range(2008,2013)]))
    
    for interval in range(1,6):
        scores[threshold][interval] = {}
        scores[threshold][interval]['scores'] = []
        y = np.array(pd.concat([development_data[year]["y"][interval] for year in range(2008,2013)]))
        
        # Binarise y data
        y_thresh = np.zeros_like(y)
        y_thresh[y>0] = 1
        
        # Balance the sample
        X_bal, y_thresh = balanced_subsample(X, y_thresh,subsample_size=1.0)
        
        scores[threshold][interval]['size'] = len(y_thresh)
        kf = KFold(n_splits=10, shuffle=True)
        for train, test in kf.split(X_bal):
            clf = RandomForestClassifier(n_estimators=150, max_depth=13)

            clf.fit(X_bal[train], y_thresh[train])
            preds = clf.predict(X_bal[test])

            new_scores = [
                sklearn.metrics.accuracy_score(y_thresh[test], preds),
                sklearn.metrics.f1_score(y_thresh[test], preds),
                np.sum(y_thresh[test]==0)/len(y_thresh[test])
            ]
            scores[threshold][interval]['scores'].append(new_scores)
        
        print(threshold, interval, len(y_thresh), np.round(np.mean(np.array(scores[threshold][interval]['scores'])[:,0]),3)
             )
        

In [ ]:
for threshold in np.arange(0.0006, 0.0017, 0.0002):
    print(threshold, '&', 
          scores[threshold][3]['size'], '&', 
          np.round(np.mean(np.array(scores[threshold][3]['scores'])[:,0]),2), 
          '$\pm$', 
          np.round(np.std(np.array(scores[threshold][3]['scores'])[:,0]),2), '&', 
          np.round(np.mean(np.array(scores[threshold][3]['scores'])[:,1]),2), 
          '$\pm$', 
          np.round(np.std(np.array(scores[threshold][3]['scores'])[:,1]),2), 
          '\\\\'
         )

In [ ]:
plt.rc('font', family='sans-serif')
plt.rc('xtick', labelsize='medium')
plt.rc('ytick', labelsize='medium')
line_styles = ['-', '--', ':']
col = 0.5
fig = plt.figure(figsize=(6,3.7))
ax = fig.add_subplot(1, 1, 1)
ax.set_title('Choosing a prediction interval, I', fontsize=13)
ax.grid()
ax.set_ylim(0.65,0.9)
#ax.set_xlim(1,5)

ax.set_ylabel('F1 score', fontsize=12)
ax.set_xlabel('Number of years in future', fontsize=12)

plt.xticks(range(1,6), range(1,6))

thresholds = ["0.0006", "0.0008", "0.0010", "0.0012", "0.0014", "0.0016"]
for i, threshold in enumerate(np.arange(0.0006, 0.0017, 0.0002)):
    y = []
    yerr = []
    for interval in range(1,6):
        s = np.array(scores[threshold][interval]['scores'])
        y.append(np.mean(s[:,1]))
        yerr.append(np.std(s[:,1]))
    
    ax.errorbar(range(1,6), y, yerr=yerr, color=str(col),  label=thresholds[i], fmt='--o')
        
    col-=0.1
    
#ax.legend(ncol=3, mode="expand")
ax.legend(fontsize=11)

In [ ]:
>>>from sklearn.ensemble import RandomForestClassifier
>>>RF_clf = RandomForestClassifier(n_estimators=10)
>>>predicted = RF_clf.predict(X_test)
>>>print '\n Here is the classification report:'
>>>print classification_report(y_test, predicted)
>>>cm = confusion_matrix(y_test, y_pred)
>>>print cm

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF_clf = RandomForestClassifier(n_estimators=10)
predicted = RF_clf.predict(X_test)
print '\n Here is the classification report:'
print classification_report(y_test, predicted)
cm = confusion_matrix(y_test, y_pred)
print cm